# I. Introduction
This notebook is meant as a comprehensive beginner's guide to creating a model using unsupervised algorithms. It explores how to cluster census tracts based on data collected from the US Census API, and includes both Python code and a running commentary.

## Table of Contents
Feature Sourcing --> what census tracts are; how to pull demographic data from the API

Exploratory Data Analysis --> get a feel for the data, combine/drop features

Feature Engineering --> process the data (transform, scale, standardize, impute)

Dimensionality Reduction --> reduce correlations & computation time

KMeans / Agglomerative --> actually assign census tracts to clusters

Cluster Profiles --> identify what each cluster's characteristics are

Validation --> use heatmap of the US to assess clustering performance


# II. Feature Sourcing

## A. Overview
A census tract is 11 numbers long and composed of three different FIPS codes: state, county and tract (the full 15 numbers is a census block, which is even more granular).

There are 74K census tracts in the entire United States, and each one contains an average of several thousand people. As a result, the physical size of census tracts range widely depending on the population density of a given area.

A full list of US state and county FIPS codes can be found here.

State FIPS Codes
There are 61 state-level FIPS codes in total because US territories also have FIPS codes.

Specifically, state codes 03, 07, 11, 14, 43, 52 or anything above 56 do not represent states and will be excluded from our example below. Alaska (02) and Hawaii (15) will also be removed.

## B. US Census API
The US Census Bureau has an API with a lot of publicly available data.

The API is open-source and provides the flexibility to pull data at various levels of aggregation.

How to find & pull data from the API
You can query a group at various levels of geographical area (state, zip code, census tract, etc.) - the full list of options can be found here
The full list of data groups available (as of Dec. 2020) can be found here
Once you have identified a group you'd like to pull, view the available reports using this link:
 https://api.census.gov/data/2019/acs/acs5/groups/{group}.html
 
(1) To download a specific report onto your computer, open the terminal and run the following (note that the query parameters will need to be changed depending on the group and geo levels you'd like to pull):

curl https://api.census.gov/data/2019/acs/acs5\?get\=NAME,{group}\&for\=tract:\*\&in\=state:\{state_fips_code}\&in\=county:\* > {group}_zip.json

(2) Open the downloaded report in a text editor by running: open {group}_zip.json

## C. Example¶
Let's pull down all the reports for group B05004 which contains data on median age.

To do this we'll need to generate:

A list of state FIPS codes for the states we'd like to pull

A list of the specific report names within each group we'd like to pull

URLs for the API calls based on the report names and state FIPs codes

We'll then run a script which makes the API calls, parses and saves the responses in a Pandas dataframe, and combines all the reports together.



"B23001_001E": Total population  
"B23001_005E": Total unemployed  
"B23001_002E": Total population in labor force  
"B23001_006E": Total unemployment in labor force  
"B23001_003E": Total male in labor force  
"B23001_007E": Total unemployment of male in labor force  
"B23001_014E": Total female in labor force  
"B23001_018E": Total unemployment of female in labor force  
"B23001_003E": White alone, in labor force  
"B23001_004E": Black or African American alone, in labor force  
"B23001_005E": American Indian and Alaska Native alone, in labor force  
"B23001_006E": Asian alone, in labor force  
"B23001_007E": Native Hawaiian and Other Pacific Islander alone, in labor force  
"B23001_008E": Some other race alone, in labor force  
"B23001_009E": Two or more races, in labor force  
"B23001_010E": White alone, unemployed, in labor force  
"B23001_011E": Black or African American alone, unemployed, in labor force  
"B23001_012E": American Indian and Alaska Native alone, unemployed, in labor force  
"B23001_013E": Asian alone, unemployed, in labor force  
"B23001_014E": Native Hawaiian and Other Pacific Islander alone, unemployed, in labor force  
"B23001_015E": Some other race alone, unemployed, in labor force  
"B23001_016E": Two or more races, unemployed, in labor force

# Run this in Anaconda base environment

In [33]:
# Import packages
import json
import requests
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [34]:
# List of US state FIPS codes
state_codes = [
    str(elem).zfill(2) 
    for elem in list(range(1, 57)) 
    if elem not in (2, 3, 7, 11, 14, 15, 43, 52)
]
print("# of State FIPS Codes:", len(state_codes))
state_codes[0:5]

# of State FIPS Codes: 48


['01', '04', '05', '06', '08']

The groups you mentioned are part of the U.S. Census Bureau’s American Community Survey (ACS) data. Here’s what each of them represents:

B05004_001E: This represents the total estimate for the median age by nativity and citizenship status by sex
B05004_002E: This represents the total estimate for the median age of males by nativity and citizenship status.
B05004_003E: This represents the total estimate for the median age of females by nativity and citizenship status.
B05004_004E: This represents the total estimate for the median age of natives by nativity and citizenship status.
B05004_005E: This represents the total estimate for the median age of native males by nativity and citizenship status.
B05004_006E: This represents the total estimate for the median age of native females by nativity and citizenship status.
B05004_007E: This represents the total estimate for the median age of foreign-born individuals by nativity and citizenship status.
B05004_008E: This represents the total estimate for the median age of foreign-born males by nativity and citizenship status
B05004_009E: This represents the total estimate for the median age of foreign-born females by nativity and citizenship status.

In [35]:
# Groups to download from census data
groups = [
        "B05004_001E", "B05004_002E", "B05004_003E", 
        "B05004_004E", "B05004_005E", "B05004_006E", 
        "B05004_007E", "B05004_008E", "B05004_009E",
]
print("# of Groups:", len(groups))
groups[0:5]

# of Groups: 9


['B05004_001E', 'B05004_002E', 'B05004_003E', 'B05004_004E', 'B05004_005E']

In [36]:
# List of URLs for API calls
url_list = [
    f"https://api.census.gov/data/2019/acs/acs5?get={group}&for=tract:*&in=state:{state_code}&in=county:*" 
    for group in groups 
    for state_code in state_codes
]
print("# of URLs:", len(url_list))
print(url_list)

# of URLs: 432
['https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:01&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:04&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:05&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:06&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:08&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:09&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:10&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:12&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:13&in=county:*', 'https://api.census.gov/data/2019/acs/acs5?get=B05004_001E&for=tract:*&in=state:16&in=county:*', 'https://api.c

In [38]:
import requests
import pandas as pd
from tqdm import tqdm

# Initialize an empty list to store DataFrames
list_of_dfs = []

# Iterate over the list of URLs
for idx, url in tqdm(enumerate(url_list)):
    # Make API call
    response = requests.get(url)
    
    # Check if the API call is successful
    if response.status_code == 200:
        # Convert JSON response to DataFrame
        df_tmp = pd.DataFrame(response.json())
        
        # Set the column names to the first row
        df_tmp.columns = df_tmp.iloc[0]
        
        # Extract the group name from the first column
        group = df_tmp.columns.tolist()[0]
        
        # Remove the first row (header row)
        df_tmp = df_tmp.iloc[1:,]
        
        # Combine state, county, and tract to create census tract identifier
        df_tmp["census_tract"] = df_tmp["state"] + df_tmp["county"] + df_tmp["tract"]
        
        # Add the group name as a new column
        df_tmp["group"] = group            
        
        # Rename the group column to "value"
        df_tmp.rename(columns = {group: "value"}, inplace=True)
        
        # Reorder the columns
        df_tmp = df_tmp[["census_tract", "group", "value"]]
        
        # Append the DataFrame to the list
        list_of_dfs.append(df_tmp)        
    else:
        # Print error message if API call is unsuccessful
        print("Index:", str(idx+1).zfill(2), "/ Error Code:", response.status_code, "/ URL:", url)

# Print the number of successful responses
print("# of Successful Responses:", len(list_of_dfs))

# Display the first DataFrame in the list
list_of_dfs[0]


432it [06:19,  1.14it/s]

# of Successful Responses: 432


,census_tract,group,value
1,01073001100,B05004_001E,39.0
2,01073001400,B05004_001E,44.3
3,01073002000,B05004_001E,34.0
4,01073003802,B05004_001E,35.8
5,01073004000,B05004_001E,52.1
...,...,...,...
1177,01077010400,B05004_001E,32.4
1178,01077011300,B05004_001E,39.4
1179,01077011602,B05004_001E,43.8
1180,01077010200,B05004_001E,40.1


In [39]:
# Combine all API calls into one dataframe
df_median_age = pd.concat(list_of_dfs)
df_median_age["value"] = df_median_age["value"].astype(float)
df_median_age = df_median_age.pivot_table(
    values="value", 
    index=df_median_age["census_tract"], 
    columns="group"
)

In [40]:
# Rename columns
df_median_age.columns = [
"all_total", "all_males", "all_females", 
"native_total", "native_males", "native_females", 
"foreign_born_total", "foreign_born_males", "foreign_born_females",
]

# Replace anything <=0 with NaN
df_median_age[df_median_age < 0] = 0
df_median_age.replace(0, np.nan, inplace=True)
df_median_age = df_median_age.reset_index()

# Confirm data types
df_median_age["census_tract"] = df_median_age["census_tract"].astype(str).str.zfill(11)
df_median_age.iloc[:, 1:] = df_median_age.iloc[:, 1:].astype(float)

df_median_age.head()

,census_tract,all_total,all_males,all_females,native_total,native_males,native_females,foreign_born_total,foreign_born_males,foreign_born_females
0,01001020100,38.9000,36.8000,40.1000,38.7000,36.6000,39.9000,43.2000,NaN,NaN
1,01001020200,41.3000,34.1000,44.6000,41.2000,34.1000,44.2000,NaN,NaN,NaN
2,01001020300,37.6000,37.2000,40.1000,37.3000,37.2000,38.1000,57.5000,59.1000,57.5000
3,01001020400,45.8000,42.5000,47.9000,47.3000,45.1000,47.8000,36.7000,36.6000,61.4000
4,01001020500,34.9000,32.5000,35.9000,34.0000,33.1000,34.4000,43.9000,24.9000,44.8000


## C. Summary
Domain knowledge is critical in deciding what types of features you want to include in your initial dataset. Depending on what problem you're trying to solve you will naturally be interested in enriching your data with the most relevant features. US census data focuses on statistics related to people but does not have much industry-specific information, so you'll need to extract and append that data from other sources.

For example, if you are interested in the US housing market and differences in pricing, you will want to add features related to properties such as number of beds/baths/rooms/stories, square footage, construction type, exterior material, quality grade, year built, replacement cost, etc.

We'll be limiting ourselves to using data from the US census in this exercise. Given that we aren't including other features in addition to the census data our algorithms will cluster based on general demographic information only. To that end we pulled reports from several groups using the methodology outlined above, and combined them into the attached dataset.



# III. Exploratory Data Analysis (EDA)
We aim to cluster census tracts with a reasonable level of success. We'll touch on a couple of ways to define "success" in this context - and the pros / cons of each method - later in the exercise. As with any data science research project the first step is to get an understanding of what our data looks like: which features it contains, what those features look like, and if we can identify features most in need of further manipulation.

Let's start by reading in our dataset as a Pandas dataframe, with the rows representing census tracts and the columns representing various features from the US census API.

In [41]:
# Import packages
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", lambda x: "%.4f" % x)
plt.style.use("ggplot")
%matplotlib inline

In [43]:
# Import the data file
import os

cwd = os.getcwd()
filepath = cwd + "/data/acs_demographic_data_by_census_tract.csv"
df = pd.read_csv(filepath)


In [44]:

# All columns in lower case
df.columns = map(str.lower, df.columns)

# Census tract 11 characters long
df["census_tract"] = df["census_tract"].astype(str).str.zfill(11)

print(df.shape)
df.head()

(72050, 50)


,state,census_tract,total_homes,total_owned,pct_owned_of_total,15-34_pct_of_owned,35-64_pct_of_owned,65+_pct_of_owned,2017+_pct_of_owned,2015-16_pct_of_owned,2010-14_pct_of_owned,2000-09_pct_of_owned,1990-99_pct_of_owned,1989-_pct_of_owned,median_age_all_total,median_age_all_males,median_age_all_females,median_age_native_total,median_age_native_males,median_age_native_females,median_age_foreign_born_total,median_age_foreign_born_males,median_age_foreign_born_females,median_age_workers_total,median_age_workers_males,median_age_workers_females,total_population,total_income,total_income_per_cap,avg_commute_in_minutes,pct_voting_age_citizens,pct_employed,pct_men,pct_poverty_all,pct_poverty_child,field_pct_professional,field_pct_service,field_pct_office,field_pct_construction,field_pct_production,commute_pct_drive,commute_pct_carpool,commute_pct_transit,commute_pct_walk,commute_pct_other,commute_pct_work_from_home,work_pct_private,work_pct_public,work_pct_self_employed,work_pct_unemployed
0,AL,01001020100,765.0000,570,0.7451,0.1017,0.6491,0.2492,0.0088,0.0298,0.1210,0.4281,0.2860,0.1263,40.5000,37.7000,43.0000,40.3000,36.6000,44.1000,43.5000,56.2000,NaN,43.6000,39.5000,46.9000,1845.0000,67826.0000,33018.0000,25.0000,0.7626,0.4775,0.4873,0.1070,0.2080,0.3850,0.1560,0.2280,0.1080,0.1240,0.9420,0.0330,0.0000,0.0050,0.0000,0.0210,0.7420,0.2120,0.0450,0.0460
1,AL,01001020200,719.0000,464,0.6453,0.0732,0.5970,0.3298,0.0237,0.0151,0.1573,0.2996,0.1121,0.3922,42.0000,34.6000,43.9000,41.7000,34.6000,43.8000,55.5000,NaN,55.9000,43.9000,44.8000,43.4000,2172.0000,41287.0000,18996.0000,22.0000,0.7606,0.3923,0.5373,0.2240,0.3580,0.3050,0.2490,0.2290,0.0630,0.1540,0.9050,0.0910,0.0000,0.0000,0.0050,0.0000,0.7590,0.1500,0.0900,0.0340
2,AL,01001020300,1296.0000,841,0.6489,0.1962,0.5101,0.2937,0.0666,0.0809,0.0761,0.3674,0.1736,0.2354,34.6000,37.1000,34.1000,34.5000,36.9000,33.7000,57.8000,59.1000,57.6000,34.7000,37.3000,34.2000,3385.0000,46806.0000,21236.0000,23.0000,0.7326,0.4378,0.4529,0.1470,0.2110,0.2790,0.1940,0.3330,0.0990,0.0960,0.8830,0.0840,0.0000,0.0100,0.0080,0.0150,0.7330,0.2110,0.0480,0.0470
3,AL,01001020400,1639.0000,1262,0.7700,0.1244,0.4968,0.3788,0.0182,0.0848,0.1403,0.3304,0.1085,0.3178,46.4000,42.1000,49.3000,45.8000,42.0000,48.1000,50.1000,48.2000,50.4000,43.2000,41.3000,47.1000,4267.0000,55895.0000,28068.0000,26.0000,0.7633,0.4333,0.4689,0.0230,0.0170,0.2900,0.1660,0.2580,0.0910,0.1950,0.8230,0.1120,0.0000,0.0150,0.0290,0.0210,0.7580,0.1970,0.0450,0.0610
4,AL,01001020500,4174.0000,2321,0.5561,0.0870,0.6829,0.2301,0.0241,0.0698,0.1478,0.5174,0.1840,0.0569,36.4000,35.8000,37.2000,35.9000,35.8000,36.1000,37.9000,37.1000,56.7000,39.2000,40.1000,38.4000,9965.0000,68143.0000,36905.0000,21.0000,0.7254,0.4804,0.5072,0.1220,0.1790,0.4880,0.1380,0.2050,0.0350,0.1340,0.8690,0.1120,0.0000,0.0080,0.0030,0.0070,0.7140,0.2410,0.0450,0.0230


Our dataset contains 50 columns, the first two of which are state and census tract (dtype="object"). It also has a row for every census tract except for ~2K of them which are located in US territories or Washington D.C.

The remaining 48 features are numerical (dtype="float") with various ranges and distributions, but we can already see opportunities for combining and/or removing some of them.

# A. States Selection
In this exercise we are not interested in looking at low-density, relatively homogeneous states in Middle America. Instead we'd like to identify census tract similarity and diversity based on the features in our dataset. We want to focus on states which are not exclusively rural but include big cities (such as California, New York or Texas) as well.

Let's identify the states with the most census tracts and focus on the top 10.

In [ ]:
num_states_to_keep = 10

num_tracts_df = df["state"].value_counts(normalize=False).reset_index()
pct_tracts_df = df["state"].value_counts(normalize=True).reset_index()
tracts_df = pd.merge(num_tracts_df, pct_tracts_df, how="inner", on="index")

tracts_df.columns = ["state", "num_tracts", "pct_tracts"]
tracts_df["cumsum_num_tracts"] = tracts_df["num_tracts"].cumsum()
tracts_df["cumsum_pct_tracts"] = tracts_df["pct_tracts"].cumsum()

tracts_df.head(num_states_to_keep)


In [ ]:
# List of states
states_to_keep = tracts_df["state"].head(num_states_to_keep).values
states_to_keep

In [ ]:
# Dataset only includes census tracts from "dense" states
df = df[df["state"].isin(states_to_keep)]
print(df.shape)

We see that just 10 states contain more than half the census tracts.
 Although here we'll limit our clustering to these 10 states, in theory wecould cluster however we'd like and see what the output is (e.g. just one state, all 50 states, anything in between).

## B. Combine & Drop Features

In [ ]:
df["2010+_pct_of_owned"] = df["2010-14_pct_of_owned"] + df["2015-16_pct_of_owned"] + df["2017+_pct_of_owned"]
df["commute_pct_car"] = df["commute_pct_drive"] + df["commute_pct_carpool"]
df["work_pct_other"] = df["work_pct_self_employed"] + df["work_pct_unemployed"]
df["total_home_to_pop_ratio"] = df["total_population"] / df["total_homes"]
df["total_income_to_per_cap_ratio"] = df["total_income"] / df["total_income_per_cap"]
df["median_age_males_females_pct_diff"] = (df["median_age_all_males"] - df["median_age_all_females"]) / df["median_age_all_females"]
df["median_age_males_females_pct_diff"] = df["median_age_males_females_pct_diff"] + abs(df["median_age_males_females_pct_diff"].min())

age_cols_to_drop = [c for c in df.columns if c[:10] == "median_age" and c[-8:] != "pct_diff" and c != "median_age_all_total"]
df.drop(age_cols_to_drop, axis=1, inplace=True)

df.drop(["2010-14_pct_of_owned", "2015-16_pct_of_owned", "2017+_pct_of_owned", "2000-09_pct_of_owned", "1990-99_pct_of_owned", "1989-_pct_of_owned"], axis=1, inplace=True)
df.drop(["commute_pct_drive", "commute_pct_carpool", "commute_pct_transit", "commute_pct_walk", "commute_pct_other", "commute_pct_work_from_home"], axis=1, inplace=True)
df.drop(["work_pct_public", "work_pct_self_employed", "work_pct_unemployed"], axis=1, inplace=True)
df.drop(["total_population", "total_owned", "total_income_per_cap"], axis=1, inplace=True)
df.drop(["field_pct_service", "field_pct_construction", "field_pct_production"], axis=1, inplace=True)
df.drop(["pct_poverty_all", "pct_poverty_child"], axis=1, inplace=True)
df.drop(["65+_pct_of_owned"], axis=1, inplace=True)

In [ ]:
# Update features list
features = [
    "total_homes", "total_home_to_pop_ratio", "total_income_to_per_cap_ratio", "pct_owned_of_total", "pct_men",
    "pct_voting_age_citizens", "pct_employed", "avg_commute_in_minutes", "commute_pct_car", "work_pct_private", 
    "work_pct_other", "field_pct_professional", "field_pct_office", "median_age_males_females_pct_diff",
    "median_age_all_total", "15-34_pct_of_owned", "35-64_pct_of_owned", "2010+_pct_of_owned",
]
df = df[["state", "census_tract"] + features]


We can get a better sense of what these features look like by visualizing them. We'll use histograms and boxplots to understand their distributions - mean, median, min, max, outliers, skewness, etc. - and a correlation matrix to determine the dependencies between them.

## C. Histogram

In [ ]:
def plot_hist(df, features, figsize=(35, 70), num_rows=15, num_cols=5, color="#ff0083", num_bins=25, alpha=0.8, lower=0.025, upper=0.975):
    fig, ax = plt.subplots(figsize=figsize)
    for feature in features:
        df_tmp = df.copy()
        ax1 = plt.subplot(num_rows, num_cols, features.index(feature) + 1)
        df_tmp[feature] = df_tmp[feature].clip(df_tmp[feature].quantile(lower), df_tmp[feature].quantile(upper))
        mean = df_tmp[feature].dropna().mean()
        median = df_tmp[feature].dropna().median()
        
        h1 = df_tmp[feature].hist(bins=num_bins, alpha=alpha, align="left", label=f"{feature} Freq Dist", color=color,
            weights=np.ones_like(df_tmp[feature].dropna()) / len(df_tmp[feature].dropna()))
        
        l1 = ax1.axvline(x=round(mean, 4), ymax=1, alpha=alpha, linestyle="dashed", label=f"{feature} Freq Mean", color="#D3D3D3")
        
        l2 = ax1.axvline(x=round(median, 4), ymax=1, alpha=alpha, linestyle="dashed", label=f"{feature} Freq Median", color="#228B22")
        
        plt.title(f"{feature}", fontsize=20, fontweight="bold")
        ax1.title.set_position([0.5, 1.05])
        ax1.spines["right"].set_visible(False)
        ax1.spines["top"].set_visible(False)
        plt.legend(loc=1, frameon=False)
        plt.xlim(df_tmp[feature].min(), df_tmp[feature].max())
        plt.ylim(0, 0.3)
        plt.grid(False)
        ax1.set_yticklabels(["{:,.0%}".format(x) for x in ax1.get_yticks()])
        
    plt.tight_layout()
    plt.show()

##  D Boxplots

In [ ]:
def plot_box(df, features, figsize=(35, 70), num_rows=15, num_cols=5, show_outliers=True):
    fig, ax = plt.subplots(figsize=figsize)
    for feature in features:
        df_tmp = df.copy()
        ax1 = plt.subplot(num_rows, num_cols, features.index(feature) + 1)
        plt.title(str(feature), fontsize=16, fontweight="bold", y=1.025)
        
        bp = plt.boxplot([df[(df[feature] >= 0)][feature]], sym="g.", showfliers=show_outliers, showmeans=True,
                         medianprops = dict(linestyle="-", linewidth=2, color="darkblue"))
        
        for flier in bp["fliers"]:
            flier.set(marker="o", color="y", alpha=0.1)
        
    plt.tight_layout()
    plt.show()

## E. Correlation Matrices

In [ ]:
def plot_correl_matrix(df, features, correl_figsize=(30, 20)):
    plt.figure(figsize=correl_figsize)
    matrix = np.triu(df[features].corr())
    sns.heatmap(df[features].corr(), annot=False, cmap="RdYlGn", mask=matrix)
    
    plt.tight_layout()
    plt.show()

## F. Plot Graphs

In [ ]:
\
def plot_graphs(df, features):  
    plot_hist(df=df, features=features)
    plot_box(df=df, features=features)
    plot_correl_matrix(df=df, features=features)
    
# Show histograms, boxplots and correlation matrix
plot_graphs(df, features)

## G. Summary

In [ ]:
# Save a copy of the dataframe with preprocessed values for later use
df_preprocessed = df.copy()

print(df.shape)
df.head()

In [ ]:
percentiles_list = [.025, .05, .25, .50, .75, .95, .975]
df.describe(percentiles=percentiles_list).apply(lambda x: x.apply("{0:.4f}".format))

## Our EDA accomplished several things:

##### 1. Reduces our feature set to 20 features

WHY? Later on we'll use an algorithm which is computationally heavy (O^2). Since we already have 72K rows i.e. census tracts we'd like to keep the number of columns i.e. features reasonable to reduce computation time. We also want to avoid the curse of dimensionality.


##### 2. Validates our features are uncorrelated

WHY? Later on we'll use a popular dimensionality reduction technique to produce orthogonal features and reduce any concerns of high-correlativity. However, some of the features in our initial dataset are highly correlated to begin with, and we can feel confident in removing them already at this stage of the process.

#####  3. Removes features with many missing values

WHY? Unlike some boosted tree algorithms which know how to account for missing values (e.g. XGBoost or LightGBM), unsupervised clustering algorithms generally don't know how to handle missing data. We therefore either need to remove NaNs or impute other values into our dataset. Later on we'll use iterative imputing to fill the remaining NaNs, but for features that are extremely sparse we'll simply remove them for the sake of this exercise.

##### 4. Focuses on features that are well distributed

WHY? All the inputs from our dataset are numerical so we prefer data that is more diverse as that will help the algorithm generate distances and clustering more effectively. Some features do have skewness and extreme outliers which will be handled by feature engineering.



## IV. Feature Engineering
There are four main preprocessing steps to perform to enable our unsupervised algorithms to effectively cluster our data:

Ensure our features are normally distributed -> Log Transform heavily right/left-skewed features

Remove outliers -> identify thresholds using the IQR Method and replace with NaNs

Handle algorithms' inherent sensitivity to scale -> apply StandardScaler to remove mean & scale to unit variance

Fill out missing values to allow for clustering -> use IterativeImputer with BayesianRidge regression (this includes nulls from original raw data & outliers replaced above)

In [ ]:
# Import packages
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

### A. Log Transform

In [ ]:
def log_transform(df, feature_skew_dict):
    for feature, skew in feature_skew_dict.items():
        assert df[feature].min() >= 0
        if skew == "right":
            df[feature] = np.log(df[feature] + 1)
        elif skew == "left":
            df[feature] = np.log((max(df[feature] + 1) - df[feature]))
            
# Apply log transform to heavily skewed features
feature_skew_dict = {
    "total_homes": "right",
    "total_home_to_pop_ratio": "right",
    "total_income_to_per_cap_ratio": "right", 
    "pct_owned_of_total": "left",
    "commute_pct_car": "left",
}

log_transform(df, feature_skew_dict)

### B. Outliers (IQR Method)

In [ ]:
def iqr_outliers(df, features):
    for feature in features:
        sorted_data = sorted(df[feature].dropna())
        q1, q3 = np.percentile(sorted_data, [25, 75])
        iqr = q3 - q1
        _lower = round(q1 - (1.5 * iqr), 4)
        _upper = round(q3 + (1.5 * iqr), 4)
        df[feature] = df[feature].apply(lambda x: np.where(x < _lower, np.nan, x))
        df[feature] = df[feature].apply(lambda x: np.where(x > _upper, np.nan, x))
        outliers_dict[feature] = {
            "min_max": {"1_min": round(df[feature].min(), 4),  "2_max": round(df[feature].max(), 4)},
            "iqr_bounds": {"1_lower": _lower, "2_upper": _upper}, 
        }

In [ ]:
# Replace outliers identified via IQR method with NaNs
outliers_dict = dict()
iqr_outliers(df, features)

# pprint(outliers_dict)

### C. StandardScaler

In [ ]:
def standard_scale(df, features):
    z_scored = StandardScaler()
    df[features] = z_scored.fit_transform(df[features])
    
    
# Scale all features to a standard normal distribution
standard_scale(df, features)

### D. IterativeImputer

In [ ]:


def impute_nans(df, features, method="iterative"):
    if method == "iterative":
            iterative_imputer = IterativeImputer(
                missing_values=np.nan,
                n_nearest_features=None,
                initial_strategy="mean",
                imputation_order="ascending",
                max_iter=10,                
                random_state=42,
                verbose=0,
            )
            df[features] = iterative_imputer.fit_transform(df[features])
    else:
        simple_imputer = SimpleImputer(
            missing_values=np.nan,
            strategy="mean",
            verbose=0,
        )
        df[features] = simple_imputer.fit_transform(df[features])
        
# Fill in NaNs with imputed values
impute_nans(df, features, method="iterative")

### E. Summary


In [ ]:
print(df.shape)
df.head()


In [ ]:
# Graph min, median & max values of each feature after they've been feature engineered
fig, ax = plt.subplots(figsize=(10, 10))
plt.plot(df.describe().columns.tolist(), df.describe().iloc[5,:], label="median", color="black", linewidth=2, marker="o")
plt.bar(df.describe().columns.tolist(), df.describe().iloc[3,:], label="minimum")
plt.bar(df.describe().columns.tolist(), df.describe().iloc[-1,:], label="maximum")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12, fontweight="bold")
plt.ylabel("Values", fontsize=16, fontweight="bold")
plt.title("Transformed, Scaled, Standardized & Imputed Values", fontsize=20, fontweight="bold")
ax.title.set_position([.5, 1.025])
plt.legend(loc="best", frameon=False)
plt.grid(False)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## V. Dimensionality Reduction

Now that our data is transformed, scaled, standardized & imputed, we'll use Principal Components Analysis (PCA) to generate principal components that are orthogonal to one another and throw away the components which don't add much by way of explainability. We'll see that the initial, low-numbered principal components explain a large amount of variance but - as we'd expect - the additional % of variance explained by later components drops as we increase the total number.

What is an acceptable amount of explainability to lose overall? A good rule of thumb is that any components beyond the ones which explain a cumulative 95% of total variance can be discarded. We'll calculate how many principal components we need to reach this threshold, and then reduce our feature set accordingly.

In [ ]:
# Import packages
from sklearn.decomposition import PCA

#### A. Identify Optimal # of PCs

In [ ]:
# Convert dataframe into numpy array (allows for faster computation)
X = df[features].values
pca = PCA(random_state=42)

pca.fit(X)
PCA(random_state=42)

df_pca = pd.DataFrame({
    "principal_component": range(1, X.shape[1]+1),
    "explained_variance": pca.explained_variance_ratio_,
    "cumsum_explained_variance": pca.explained_variance_ratio_.cumsum(),
})

df_pca.loc[-1] = 0
df_pca.sort_values(by="principal_component", inplace=True)

print(df_pca.shape)
df_pca.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(df_pca["principal_component"], df_pca["explained_variance"], marker="o", label="Individual Explained Variance")
plt.plot(df_pca["principal_component"], df_pca["cumsum_explained_variance"], marker="o", label="Cumulative Explained Variance")
plt.plot(df_pca["principal_component"], [0.95] * len(df_pca), color="black", linewidth=2, linestyle="--", label="95% Explained Variance")
ax.set_xticklabels(["{:,.0f}".format(x) for x in ax.get_xticks()])
ax.set_yticklabels(["{:,.0%}".format(x) for x in ax.get_yticks()])
plt.title("PCA Explained Variance", fontsize=20, fontweight="bold")
ax.title.set_position([.5, 1.025])
plt.xticks(range(0,21), range(0,21), fontsize=12, fontweight="bold")
plt.yticks(fontsize=12, fontweight="bold")
plt.xlabel("Principal Component Number", fontsize=16, fontweight="bold")
plt.ylabel("% of Explained Variance", fontsize=16, fontweight="bold")
plt.grid(False)
plt.legend(loc="best", frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
n_components_pca = int(df_pca["cumsum_explained_variance"].gt(0.95).idxmax())
print("# of Features Until 95% Variance is Reached:", n_components_pca)

### B. PCA Fit & Transform

In [ ]:
pca = PCA(n_components=n_components_pca, random_state=42)
X = pca.fit_transform(X)

print(X.shape)
X

## VI. KMeans Clustering

In order to determine the optimal number of clusters for the algorithm we'll use the popular Elbow Method where we calculate the Within Cluster Sum of Squares (WCSS) - also known as intertia - for each potential number of clusters we could use. This hyperparameter tuning will help us identify how many clusters to instantiate our KMeans model with.

We'll create a Scree Plot where the X-axis maps the number of clusters and the Y-axis maps the inertia. The value of x where the plot forms an "elbow" - i.e. the slope change vs. the previous number of clusters reduces the most - is what we'll use for our model.fit_predict.

In [ ]:
# Import packages
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#### A. Elbow Method (Scree Plot)

In [ ]:
# Define range of clusters to check
inertia_scores = []
silhouette_scores = []
no_of_clusters = range(2, 22)

# Calculate intertia & silhouette average for each cluster
for cluster in tqdm(no_of_clusters):
    kmeans = KMeans(n_clusters=cluster, init="k-means++", random_state=42)
    kmeans = kmeans.fit(X)
    
    inertia = kmeans.inertia_
    silhouette_avg = silhouette_score(X, kmeans.labels_)
    
    inertia_scores.append(round(inertia))
    silhouette_scores.append(silhouette_avg)

In [ ]:
# Interia scree plot
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(range(1, len(no_of_clusters)+1), inertia_scores, marker="o", linewidth=2, linestyle="--")
plt.xticks(range(1, len(no_of_clusters)+1), no_of_clusters, fontsize=12, fontweight="bold")
ax.set_yticklabels(["{:,.0f}".format(x/1000) + "K" for x in ax.get_yticks()])
plt.yticks(fontsize=12, fontweight="bold")
plt.xlabel("# of Clusters", fontsize=16, fontweight="bold")
plt.ylabel("Inertia", fontsize=16, fontweight="bold")
plt.title("Inertia Scree Plot per Cluster", fontsize=20, fontweight="bold")
ax.title.set_position([.5, 1.025])
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
slopes = [0]
slopes_pct_change = []
inertia_df = pd.DataFrame()
inertia_df["inertia"] = inertia_scores
inertia_df["n_clusters"] = inertia_df.index + 2

In [ ]:
def derivative_calc(df, x_field, y_field):
    x_values = df[x_field].values
    y_values = df[y_field].values
    for i in range(1, len(x_values)):
        (x1, y1) = (x_values[i-1], y_values[i-1])
        (x2, y2) = (x_values[i], y_values[i])
        slope = round((y2 - y1) / (x2 - x1), 4)
        slopes.append(slope)
        slopes_pct_change.append((abs(slopes[i-1]) - abs(slopes[i])) / abs(slopes[i-1]))
    df["slopes"] = slopes
    df["slopes_pct_change"] = slopes_pct_change + [0]

In [ ]:
# Define optimal number of clusters
derivative_calc(inertia_df, "n_clusters", "inertia")
n_clusters_kmeans = int(inertia_df.loc[inertia_df["slopes_pct_change"].idxmax()]["n_clusters"])
print("# of Clusters for KMeans Algorithm:", n_clusters_kmeans)

In [ ]:
inertia_df[["n_clusters", "inertia", "slopes", "slopes_pct_change"]].head(10)

### B. Model Fit & Predict¶

In [ ]:
kmeans = KMeans(n_clusters=n_clusters_kmeans, init="k-means++", random_state=42)
y_kmeans = kmeans.fit_predict(X)

def cluster_cnts(predictions, algorithm):
    
    unique, counts = np.unique(predictions, return_counts=True)
    cluster_cnts_df = pd.DataFrame(counts)
    cluster_cnts_df["ratio"] = round(100 * cluster_cnts_df[0] / cluster_cnts_df[0].sum(), 4)
    cluster_cnts_df = cluster_cnts_df.reset_index()
    cluster_cnts_df.columns = ["cluster", "count", "ratio"]
    
    print(f"Breakdown of Census Tracts in Each {algorithm} Cluster")
    return cluster_cnts_df

cluster_cnts(y_kmeans, "KMeans")

### C. Silhouette Score

In [ ]:
silhouette_kmeans_euclidean = round(silhouette_score(X, y_kmeans, metric="euclidean"), 4)
silhouette_kmeans_manhattan = round(silhouette_score(X, y_kmeans, metric="manhattan"), 4)
print("Silhouette Score KMeans Euclidean:", silhouette_kmeans_euclidean, "\nSilhouette Score KMeans Manhattan:", silhouette_kmeans_manhattan)

The silhouette score is a metric between [-1, 1] that incorporates the mean intra-cluster distance and mean nearest-cluster distance. A score of 1 means the clusters are dense and nicely separated i.e. the mean intra-cluster distance is large and the mean nearest-cluster distance is small. A score of 0 means the cluster boundaries may be overlapping i.e the mean intra-cluster distance is small and the mean nearest-cluster distance is large. A score of -1 means that samples likely got assigned to the wrong clusters, and we need to revisit the data.


It is inherently difficult to assess the performance of unsupervised algorithms since by definition we don't know what the truth value we're looking to identify actually is. Silhouette score is a good option, but suffers from a couple of major drawbacks. One is that it's problematic to compare between different algorithms using it as a benchmark, so a lower silhouette score in, say, Agglomerative vs. KMeans is not necessarily conclusive. Additionally, increasing the volume of the data will almost always decrease this score, since there is more variability in the data and hence a lower likelihood for a "perfect" clustering to be possible.


Out dataset contains tens of thousands of rows, making a high silhouette score a near impossibility (even if we had more and/or better features to work with). For this exercise, however, we have another recourse to assess the algorithms' performance. The clustering is agnostic to the meaning of a census tract; all it knows are the associated features. Since we have a intuitive idea of what areas in the states we selected should be "similar", we can visualize the census tracts on a map, color-code them according to cluster, and see if the result aligns with what we'd expect to find. We will do this later on.

### D. Example

Let's choose two random principal components to graph to assess if clearly defined clusters are evident (since we can only visualize in 2D or 3D we cannot "see" the clusters in this way for all 15 Principal Components).

In [ ]:
def plot_scatter(X, y, features_to_compare, algorithm, is_kmeans=True):
    fig, ax = plt.subplots(figsize=(16, 10))
    plt.scatter(X[y == 0, features_to_compare[0]], X[y == 0, features_to_compare[1]], s=25, c="red", alpha=0.8, label="Cluster 0")
    plt.scatter(X[y == 1, features_to_compare[0]], X[y == 1, features_to_compare[1]], s=25, c="yellow", alpha=0.6, label="Cluster 1")
    plt.scatter(X[y == 2, features_to_compare[0]], X[y == 2, features_to_compare[1]], s=25, c="green", alpha=0.4, label="Cluster 2")
    plt.scatter(X[y == 3, features_to_compare[0]], X[y == 3, features_to_compare[1]], s=25, c="blue", alpha=0.2, label="Cluster 3")
    
    if is_kmeans:
        plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], s=200, c="black", marker="^", label="Centroid")
    
    plt.title(f"PCA Components {features_to_compare[0]} vs {features_to_compare[1]} {algorithm}", fontsize=20, fontweight="bold")
    ax.title.set_position([.5, 1.025])
    plt.legend(loc="best", frameon=False)
    plt.grid(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_scatter(X=X, y=y_kmeans, features_to_compare=[1, 11], algorithm="KMeans", is_kmeans=True)

### VII. Agglomerative Clustering

A major advantage of Agglomerative clustering over KMeans is that you don't have to provide the number of clusters as a hyperparameter. By graphing a dendrogram - which is basically a visualization of the clustering technique used by this algorithm - we can determine this number via the following:

Identify which section of the dendrogram has the largest vertical distance where there is no new "branching off" that occurs
Draw a horizontal line through that section of the dendrogram from one end to the other
Count the number of vertical lines that intersect the horizontal line
Another advantage of Agglomerative is that it has a unique result; no matter how many times we run the algorithm it always produces the same dendrogram (assuming no changes are made to the data). This is in contrast to KMeans where without a random state defining the initial centroid locations the final results could vary with each run.

In [ ]:
# Import packages
from scipy.cluster import hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

#### A. Dendrogram

In [ ]:
plt.figure(figsize=(30, 30))
dendrogram = sch.dendrogram(sch.linkage(X, method="ward"))

The dendrogram indicates our data is most naturally divided into two clusters, as the distance between 250 and 350 is the largest vertical distance uninterrupted by a split and it contains just two trees within it. This makes sense for a couple of reasons:

Our original features are not very interesting in the sense that generic population stats don't have any domain-related "nuance"
It is natural to divide geographic areas into two broad categories: urban and rural
The dendrogram's recommendation notwithstanding, we'll use four clusters here as well to be consistent with our choice for KMeans and to provide more color in our final analysis.

In [ ]:
# Define optimal number of clusters based on dendrogram
n_clusters_agglom = 4
print("# of Clusters for Agglomerative Algorithm:", n_clusters_agglom)

#### B. Model Fit & Predict


In [ ]:
agglom = AgglomerativeClustering(n_clusters=n_clusters_agglom, affinity="euclidean", linkage="ward")
y_agglom = agglom.fit_predict(X)
cluster_cnts(y_agglom, "Agglomerative")

#### C. Silhouette Score

In [ ]:
silhouette_agglom_euclidean = round(silhouette_score(X, y_agglom, metric="euclidean"), 4)
silhouette_agglom_manhattan = round(silhouette_score(X, y_agglom, metric="manhattan"), 4)
print("Silhouette Score Agglomerative Euclidean:", silhouette_agglom_euclidean, "\nSilhouette Score Agglomerative Manhattan:", silhouette_agglom_manhattan)


#### D. Example

Unlike the KMeans algorithm, the Agglomerative algorithm's scatter plot does not include centroids. This is because the algorithm doesn't use centroids to determine its clusters.

In [ ]:
plot_scatter(X=X, y=y_agglom, features_to_compare=[1, 11], algorithm="Agglomerative", is_kmeans=False)

### VIII. Cluster Profiles

Now that we have our predictions, we'd like to include the prediction responses in our preprocessed Pandas dataframe, and try to understand what is unique about each cluster. We can then create a "profile" for each cluster, and visualize the census tracts on a heatmap of the US.

#### A. Calculate Average Values for each Cluster

In [ ]:
df_preprocessed["kmeans_pred"] = y_kmeans
df_preprocessed["agglom_pred"] = y_agglom
df_all_kmeans_avgs = df_preprocessed.groupby("kmeans_pred").mean().reset_index()[["kmeans_pred"] + features]
df_all_agglom_avgs = df_preprocessed.groupby("agglom_pred").mean().reset_index()[["agglom_pred"] + features]
df_all_kmeans_avgs

In [ ]:
df_all_agglom_avgs

### B. Plot Graphs

In [ ]:
def plot_bar(df, group_col):
    fig, ax = plt.subplots(figsize=(30, 30))
    for idx, f in enumerate(df.columns[1:]):
        ax1 = plt.subplot(4, 5, idx+1)
        plt.bar(df[group_col], df[f], alpha=0.8)
        plt.ylim(df[f].min() * 0.8, df[f].max() * 1.2)
        plt.title(f, fontsize=16, fontweight="bold")
        plt.xticks(rotation=0)
        plt.grid(False)
    
        labels = round(df[f], 2).values.tolist()
        for rect, label in zip(ax1.patches, labels):
            height = rect.get_height()
            ax1.text(rect.get_x() + rect.get_width() / 2, height + 0.05, label, ha="center", va="bottom")
    
    plt.tight_layout()
    plt.show()
plot_bar(df_all_kmeans_avgs, "kmeans_pred")

Perhaps the most interesting insight here is that some features are more meaningful than others. For example, the feature pct_men seems relatively unimpactful, while the feature total_income_to_per_cap_ratio clusters two groups very differently than the other two. Unsupervised clustering algorithms such as KMeans assume equal weight to all the features they are provided with as they are simply calculating distance irrespective of the "importance" a feature actually has. This can negatively affect its desired performance when some features should be more heavily accounted for than others. These results indicate that we should rethink how critical some of the 20 features we used really are.

Let's also get a sense of each cluster's characteristics by calculating the rank for each feature and comparing between them.

### C. Summary

In [ ]:
# Replace mean value with rank relative to all the clusters (1 = lowest, 4 = higheest)
for col in df_all_kmeans_avgs.columns:
    df_all_kmeans_avgs[col] = df_all_kmeans_avgs[col].rank()
    
# Show as Pandas dataframe
cluster_profiles_df = df_all_kmeans_avgs.T
cluster_profiles_df = cluster_profiles_df.astype(int).reset_index()
cluster_profiles_df.columns = ["Features", "Cluster 0", "Cluster 1", "Cluster 2", "Cluster 3"]

print(cluster_profiles_df.iloc[:, 1:].sum())
cluster_profiles_df.iloc[1:, :]

In [ ]:
#We can do the heat map in this case

### X. Conclusion

In this notebook we built a pipeline designed to address every step of creating a clustering model, from gathering and exploring the data to manipulating and refining it to actually generating the clusters to (finally) understanding and validating them. We'd like to stress that the actual results of this exercise would be greatly improved by significantly expanding our initial feature set from both a quantitative (the silhouette scores could be higher) and qualitative (the heatmap has patches which seem "off") perspective. However, our main goal was to demonstrate a methodology which can be adapted and used in various practical applications.